In [1]:
import igraph as ig
import numpy as np
import louvain
import time

In [2]:
print("Construct the graph from the txt file (edgelist)")

input = np.loadtxt("Data/ElistPowerUS.txt", dtype='i', delimiter=' ');
dim = input.shape;
N = input.max();
g = ig.Graph();
g.add_vertices(N);
for i in range(0, dim[0]):
    g.add_edges([(input[i,0]-1,input[i,1]-1)]);
ig.summary(g)

Construct the graph from the txt file (edgelist)
IGRAPH U--- 4941 6594 -- 


In [3]:
print("Largest connected component")
largest = g.clusters().giant();
ig.summary(largest)

Largest connected component
IGRAPH U--- 4941 6594 -- 


In [4]:
print("Running Louvain 1 times in C++")
start_time = time.time()

part = louvain.find_partition(largest, louvain.ModularityVertexPartition);

        
print("--- %s seconds ---" % (time.time() - start_time))

print("Modularity :",part.modularity)
print("Sizes of the groups:", part.sizes())

Running Louvain 1 times in C++
--- 0.05741548538208008 seconds ---
Modularity : 0.9367274585060877
Sizes of the groups: [262, 208, 201, 200, 196, 189, 187, 184, 180, 169, 164, 154, 153, 147, 144, 139, 136, 135, 134, 131, 121, 121, 119, 108, 105, 105, 100, 85, 83, 83, 79, 78, 78, 77, 55, 49, 45, 19, 18]


In [4]:
print("Running Louvain 1000 times in C++")
start_time = time.time()

m_best =0;
part_best = louvain.find_partition(largest, louvain.ModularityVertexPartition);


for n in range(1000):
    part = louvain.find_partition(largest, louvain.ModularityVertexPartition);
    m = part.modularity;
    if m_best<m:
        m_best = m;
        part_best = part;
        

print("--- %s seconds ---" % (time.time() - start_time))

print("Modularity :",part_best.modularity)
print("Sizes of the groups:", part_best.sizes())

Running Louvain 1000 times in C++
--- 61.680736780166626 seconds ---
Modularity : 0.9378112693141412
Sizes of the groups: [200, 199, 198, 197, 197, 171, 171, 168, 166, 164, 164, 158, 155, 154, 151, 147, 143, 143, 131, 124, 123, 115, 109, 108, 105, 102, 100, 98, 97, 94, 81, 78, 72, 71, 65, 62, 55, 45, 31, 29]


In [5]:
len(part_best)

40

In [5]:
from scipy.sparse import csr_matrix
from scipy.sparse import lil_matrix

membership =part_best.membership;
H = lil_matrix((part_best.n,len(part_best.sizes())));
i = 0;
for n in membership:
    H[i,n] =1.;
    i = i+1;
    
e_list = largest.get_edgelist();
e_list = np.array(e_list);

A = lil_matrix((part_best.n,part_best.n), dtype=np.int8);

for n in range(len(e_list)):
    A[e_list[n,0],e_list[n,1]] = 1;

A = A+lil_matrix.transpose(A);
deg = A.sum(0)


L_n = lil_matrix((part_best.n,part_best.n));

for n in range(len(e_list)):
    L_n[e_list[n,0],e_list[n,1]] = -1./np.sqrt(deg[0,e_list[n,0]]*deg[0,e_list[n,1]]);
    
L_n = L_n+lil_matrix.transpose(L_n);


for a in range(part_best.n):
    L_n[a,a] = 1.;
            

T =(lil_matrix.transpose(H)*L_n*H);
d = np.array(T.diagonal());
n_cut = np.sum(d);
print('Normalized Cut')
print(n_cut)

/home/fanuel/anaconda3/lib/python3.6/site-packages/scipy/sparse/compressed.py:774: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  SparseEfficiencyWarning)


Normalized Cut
579.307921805
